# How to customize or contribute to the knowledge base

## guideline
You can design your own description language and use it with Draco or extend the existing language we use here. If you don't know where to start with the constraints, you can first set up the search space in `generate.lp` and use our `run_clingo` API to generate some recommendations. Then, you should be able to find some recommendations that should have been left out, and you can write constraints to reflect them.  

For example the following snippet shows how to use the `run_clingo` API to generate 1 recommendation.

In [2]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo
from draco.programs import define, hard, helpers, constraints, generate
from pprint import pprint

prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
)
with open("../draco/asp/examples/scatter.lp") as file:
    scatter = file.read()
for model in run_clingo(prog + scatter, 1):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)

{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'value',
 'view': [{'mark': [{'channel': 'text',
                     'encoding': [{'channel': 'text',
                                   'field': ('f', 0),
                                   'scale_type': 'linear'},
                                  {'channel': 'size',
                                   'field': ('f', 1),
                                   'scale_type': 'linear'}],
                     'scale': ('s', 1),
                     'type': 'text'}],
           'scale': [{'channel': 'text', 'type': 'linear'},
                     {'channel': 'size', 'type': 'linear'}]}]}
[attribute(number_rows,root,100), attribute((field,name),(f,0),temperature), attribute((field,type),(f,0),number), attribute((field,name),(f,1),precipitation), attribute((field,type),(f,1),number), attribute((encoding,field),(e,0),(f,0)), attribute((encoding,field),(e,1)

If you see there are too many recommendations, you can:
 * add more hard constraints
 * modify your generator and hard constraints to reduce symmetry in the search space (ex. similar recommendations with switched entity ids)


If you see too few recommendations, you can:
 * check if some of your constraints are too tight, and move them to soft constraints
 

If you see no recommendations, you might have made mistakes in the hard constraints. You can allow violations to check what are the common ones:

In [3]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo, get_violations
from draco.programs import define, hard, helpers, constraints, generate
from draco.asp_utils import blocks_to_program
from pprint import pprint

c = "".join(
    blocks_to_program(
        constraints.blocks, set(constraints.blocks.keys()) - set(["violation"])
    )
)
prog = generate.program + define.program + helpers.program + hard.program + c
with open("../draco/asp/examples/scatter.lp") as file:
    scatter = file.readlines()
scatter = ("\n").join(scatter)
for model in run_clingo(prog + scatter, 1):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    answer = [str(symbol) + ". " for symbol in model.answer_set]
    print(get_violations(answer))

{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'value',
 'view': [{'mark': [{'channel': 'shape',
                     'encoding': [{'channel': 'shape',
                                   'field': ('f', 0),
                                   'scale_type': 'categorical'},
                                  {'channel': 'size',
                                   'field': ('f', 1),
                                   'scale_type': 'categorical'}],
                     'mark_channel_discrete_or_binned': 'shape',
                     'scale': ('s', 1),
                     'type': 'rect'}],
           'scale': [{'channel': 'shape', 'type': 'categorical'},
                     {'channel': 'size', 'type': 'categorical'}]}]}
[attribute(number_rows,root,100), attribute((field,name),(f,0),temperature), attribute((field,type),(f,0),number), attribute((field,name),(f,1),precipitation), attribute((field,type),